In [1]:
# Intentar hacer que no haga falta que quien corra el notebook se descargue antes los datos
# Que se descarguen en la misma ubicación donde esté el notebook
# Leer solo los archivos '*.csv'
# Append de todos los ficheros
# Valorar si leer los archivos horarios o diarios (según tamaño)
# más interesante datos en tiempo real, para ver patrones de comportamiento a lo largo del día, fin de semana...
# pandas, matplotlib
# función para coger un csv, eliminar líneas que no interesen y convertirlo en un df
# usar esa función dentro de otra que haga un append de un número ilimitado de dfs
# El append de todo va a ser imposible
# Dejamos para el final el tema de descargar y unzipear

In [2]:
import pandas as pd
# import logging

In [3]:
data = pd.read_csv('data/Anio201810/abr_mo18.csv', sep=';')
data.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2018,4,1,2.0,V,...,2.0,V,2.0,V,3.0,V,4.0,V,3.0,V
1,28,79,4,1,28079004_1_38,2018,4,2,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
2,28,79,4,1,28079004_1_38,2018,4,3,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
3,28,79,4,1,28079004_1_38,2018,4,4,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
4,28,79,4,1,28079004_1_38,2018,4,5,2.0,V,...,2.0,V,2.0,V,4.0,V,5.0,V,4.0,V


In [4]:
data.shape

(4490, 56)

In [5]:
def get_stacked_dataframe(dataframe, cols_to_drop, cols_remain):
    '''This function applies the pandas stack method to make data that is spread in columns collapse in a single column.
    First drops the columns that would not let the stack work properly.
    Then sets the columns that do not have to be stacked as the index.
    Applies stack method. Finally, resets index.
    '''
    dataframe = dataframe.drop(columns=cols_to_drop)
    dataframe = dataframe.set_index(cols_remain)
    dataframe = dataframe.stack().reset_index()
    return dataframe

In [6]:
def add_last_col_to_df(df1, df2):
    ''' Adds the last column from a dataframe to another dataframe with the same number of rows'''
    df1['new_col'] = df2.iloc[:,-1]
    return df1

In [7]:
def time_transformation(hstring):
    '''Transforms this kind of data: 'H01' into 1'''
    return int(hstring[-2:])

In [8]:
def get_reshaped_df(dataframe):
    '''Gets a df, keeps just the NO2 info, splits it into 2 dataframes, each of them with one of the
    columns that we want to stack, joins them into a single dataframe, renames columns and formats HORA column.
    The result is a much more easy to use dataframe'''
    print(f'Dataframe shape: {dataframe.shape}')
    
    print('Keeping just NO2 data')
    dataframe = dataframe[dataframe['MAGNITUD'] == 8].drop(columns=['MAGNITUD'])
    cols_dimensiones = ['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA']
    
    print('Stacking dataframes')
    df_h = get_stacked_dataframe(
        dataframe,
        cols_remain=cols_dimensiones,
        cols_to_drop=[col for col in list(dataframe.columns) if col[0] == 'V']
    )
    
    df_v = get_stacked_dataframe(
        dataframe,
        cols_remain=cols_dimensiones,
        cols_to_drop=[col for col in list(dataframe.columns) if col[0] == 'H']
    )
    
    print('Joining dataframes')
    final_df = add_last_col_to_df(df_h, df_v)
    
    print('Renaming columns')
    final_df = final_df.rename(columns={'level_7': 'HORA', 0: 'NIVEL_NO2', 'new_col': 'VALIDADO'})
    
    print('Formatting HORA column')
    final_df['HORA'] = final_df['HORA'].apply(time_transformation)
    
    print(f'Final dataframe shape: {final_df.shape}')
    
    return final_df

In [9]:
reshaped_df = get_reshaped_df(data)
reshaped_df.head()

Dataframe shape: (4490, 56)
Keeping just NO2 data
Stacking dataframes
Joining dataframes
Renaming columns
Formatting HORA column
Final dataframe shape: (17256, 10)


,PROVINCIA,MUNICIPIO,ESTACION,PUNTO_MUESTREO,ANO,MES,DIA,HORA,NIVEL_NO2,VALIDADO
0,28,79,4,28079004_8_8,2018,4,1,1,21.0,V
1,28,79,4,28079004_8_8,2018,4,1,2,19.0,V
2,28,79,4,28079004_8_8,2018,4,1,3,17.0,V
3,28,79,4,28079004_8_8,2018,4,1,4,24.0,V
4,28,79,4,28079004_8_8,2018,4,1,5,17.0,V
